# Projet 4

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import multiprocessing as mp

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [11]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    kim = requests.get(url)
    kim = kim.content.decode('utf-8')
    soup = BeautifulSoup(kim)
    categorie =soup.find_all("h2")
    prices = {}
    for cat in categorie :
        c = cat.text
        dsub={}
        for i in cat :
            price = i.find_next("span").text.split()[0]
            storage = i.find_next('ul').select('li')[3].text.split()[0]
            database = int(i.find_next('ul').select('li')[4].text.split()[0])
            dsub["price"]= price
            dsub["storage"]= storage
            dsub["databases"]= database
            prices[c] = dsub
    return (prices)

In [12]:
get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

In [13]:
get_prices_from_url(URL_PAGE3)


{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [5]:
def extract_beer_infos(url):
    bw = requests.get(url)
    bw = bw.content.decode('utf-8')
    sp = BeautifulSoup(bw)
    name = sp.find("title").text.split("  |")[0]
    n = re.compile('<div class="stars" data-percent="([^<]+)"></div>')
    note = n.search(bw)
    nn =int(note.group(1))
    p = re.compile('<span class="price">([^<]+)€</span>')
    price = p.search(bw)
    pp = float(price.group(1).replace(",","."))
    volume = int(sp.find('dd', {"class" : "small-6 medium-9 columns js-beer-volume"}).attrs["data-volume"].split()[0])
    infos = {
        'name': name,
        'note': nn,
        'price': pp,
        'volume': volume,
    }
    return infos

In [6]:
extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33")

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [7]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [8]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    response = requests.get(URL_BEERLIST_FRANCE)
    bwf = response.json()
    items = bwf["items"]
    beer_pages=[]
    for i, j in enumerate(items):
        page = bwf["items"][i]["contentReference"]
        beer_pages.append('https://www.beerwulf.com'+page)
    
    # Sequential version (slow):
    beers = []
    for beer in beer_pages :
        beers.append(extract_beer_infos(beer))
    
    # Parallel version (faster):
    # beers = []
    # p = mp.Pool(len(beer_pages))
    # beers = p.map(extract_beer_infos, beer_pages)
    
    return beers

In [9]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [14]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.003s

OK
